In [1]:
import pandas as pd
import numpy as np
import re

import pickle

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import make_scorer, mean_absolute_error, r2_score, mean_squared_error, f1_score, classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler
from catboost import Pool, cv, CatBoostRegressor, CatBoostClassifier
from skopt import BayesSearchCV
from skopt.space import Integer, Real
from skopt.callbacks import DeadlineStopper, VerboseCallback, DeltaXStopper, DeltaYStopper
from modAL.models import ActiveLearner
import seaborn as sns

%matplotlib inline

pd.set_option('display.max_columns', 50)

In C:\Users\tuman\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\tuman\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\tuman\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\tuman\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\tuman\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

In [3]:
train_data = pd.read_csv('./preprocessed_data/train_data.csv')
test_data = pd.read_csv('./preprocessed_data/test_data.csv')

In [4]:
train_data = train_data[(~train_data['address'].isnull())]
cols = [col for col in list(train_data) if 'price_type' not in col and col != 'id']

train_data_1 = train_data[train_data['price_type'] == 1]
train_data_0 = train_data[train_data['price_type'] == 0].sample(train_data_1.shape[0])
train_data_strip = pd.concat([train_data_0, train_data_1])

In [5]:
data_to_predict_type = train_data.drop(train_data_strip.index.values, axis=0)

In [6]:
x_train, x_valid, y_train, y_valid = train_test_split(train_data_strip[cols], 
                                                      train_data_strip['price_type'], 
                                                      test_size=0.1, 
                                                      random_state=42,
                                                      stratify=train_data_strip['price_type'])

learner = ActiveLearner(
                        estimator=CatBoostClassifier(
                                cat_features=['osm_city_nearest_name', 'day', 'realty_type', 'region'],
                                text_features=['text_floor', 'address'],
                                loss_function='Logloss',
                                eval_metric='AUC',
                                task_type='CPU',
                                text_processing = {
                                    "tokenizers" : [{
                                        "tokenizer_id" : "Space",
                                        "separator_type" : "ByDelimiter",
                                        "delimiter" : " "
                                    }],

                                    "dictionaries" : [{
                                        "dictionary_id" : "BiGram",
                                        "token_level_type": "Letter",
                                        "max_dictionary_size" : "50000",
                                        "occurrence_lower_bound" : "1",
                                        "gram_order" : "2"
                                    }
                                    ],

                                    "feature_processing" : {
                                        "default" : [
                                                {
                                                "dictionaries_names" : ["BiGram"],
                                                "feature_calcers" : ["BoW"],
                                                "tokenizers_names" : ["Space"]
                                            },
                                                {
                                            "dictionaries_names" : ["BiGram"],
                                            "feature_calcers" : ["NaiveBayes"],
                                            "tokenizers_names" : ["Space"]
                                        },{
                                            "dictionaries_names" : ["BiGram"],
                                            "feature_calcers" : ["BM25"],
                                            "tokenizers_names" : ["Space"]
                                        },
                                        ],
                                    }
                                }
                            ),
    X_training=x_train, y_training=y_train
)

Learning rate set to 0.024402
0:	total: 299ms	remaining: 4m 58s
1:	total: 441ms	remaining: 3m 40s
2:	total: 589ms	remaining: 3m 15s
3:	total: 734ms	remaining: 3m 2s
4:	total: 860ms	remaining: 2m 51s
5:	total: 1.01s	remaining: 2m 47s
6:	total: 1.13s	remaining: 2m 40s
7:	total: 1.26s	remaining: 2m 36s
8:	total: 1.39s	remaining: 2m 33s
9:	total: 1.51s	remaining: 2m 30s
10:	total: 1.67s	remaining: 2m 29s
11:	total: 1.81s	remaining: 2m 28s
12:	total: 1.97s	remaining: 2m 29s
13:	total: 2.13s	remaining: 2m 30s
14:	total: 2.26s	remaining: 2m 28s
15:	total: 2.39s	remaining: 2m 26s
16:	total: 2.51s	remaining: 2m 25s
17:	total: 2.67s	remaining: 2m 25s
18:	total: 2.8s	remaining: 2m 24s
19:	total: 2.96s	remaining: 2m 24s
20:	total: 3.1s	remaining: 2m 24s
21:	total: 3.23s	remaining: 2m 23s
22:	total: 3.39s	remaining: 2m 24s
23:	total: 3.55s	remaining: 2m 24s
24:	total: 3.68s	remaining: 2m 23s
25:	total: 3.81s	remaining: 2m 22s
26:	total: 3.94s	remaining: 2m 22s
27:	total: 4.07s	remaining: 2m 21s
28:

235:	total: 32.4s	remaining: 1m 44s
236:	total: 32.5s	remaining: 1m 44s
237:	total: 32.6s	remaining: 1m 44s
238:	total: 32.8s	remaining: 1m 44s
239:	total: 32.9s	remaining: 1m 44s
240:	total: 33s	remaining: 1m 43s
241:	total: 33.1s	remaining: 1m 43s
242:	total: 33.2s	remaining: 1m 43s
243:	total: 33.4s	remaining: 1m 43s
244:	total: 33.5s	remaining: 1m 43s
245:	total: 33.6s	remaining: 1m 43s
246:	total: 33.8s	remaining: 1m 42s
247:	total: 33.9s	remaining: 1m 42s
248:	total: 34s	remaining: 1m 42s
249:	total: 34.2s	remaining: 1m 42s
250:	total: 34.3s	remaining: 1m 42s
251:	total: 34.5s	remaining: 1m 42s
252:	total: 34.6s	remaining: 1m 42s
253:	total: 34.7s	remaining: 1m 41s
254:	total: 34.8s	remaining: 1m 41s
255:	total: 35s	remaining: 1m 41s
256:	total: 35.1s	remaining: 1m 41s
257:	total: 35.2s	remaining: 1m 41s
258:	total: 35.4s	remaining: 1m 41s
259:	total: 35.5s	remaining: 1m 40s
260:	total: 35.6s	remaining: 1m 40s
261:	total: 35.8s	remaining: 1m 40s
262:	total: 35.9s	remaining: 1m 40

466:	total: 1m 3s	remaining: 1m 12s
467:	total: 1m 3s	remaining: 1m 12s
468:	total: 1m 3s	remaining: 1m 12s
469:	total: 1m 3s	remaining: 1m 12s
470:	total: 1m 4s	remaining: 1m 11s
471:	total: 1m 4s	remaining: 1m 11s
472:	total: 1m 4s	remaining: 1m 11s
473:	total: 1m 4s	remaining: 1m 11s
474:	total: 1m 4s	remaining: 1m 11s
475:	total: 1m 4s	remaining: 1m 11s
476:	total: 1m 4s	remaining: 1m 11s
477:	total: 1m 4s	remaining: 1m 10s
478:	total: 1m 5s	remaining: 1m 10s
479:	total: 1m 5s	remaining: 1m 10s
480:	total: 1m 5s	remaining: 1m 10s
481:	total: 1m 5s	remaining: 1m 10s
482:	total: 1m 5s	remaining: 1m 10s
483:	total: 1m 5s	remaining: 1m 10s
484:	total: 1m 5s	remaining: 1m 9s
485:	total: 1m 5s	remaining: 1m 9s
486:	total: 1m 6s	remaining: 1m 9s
487:	total: 1m 6s	remaining: 1m 9s
488:	total: 1m 6s	remaining: 1m 9s
489:	total: 1m 6s	remaining: 1m 9s
490:	total: 1m 6s	remaining: 1m 9s
491:	total: 1m 6s	remaining: 1m 8s
492:	total: 1m 6s	remaining: 1m 8s
493:	total: 1m 7s	remaining: 1m 8s
49

696:	total: 1m 34s	remaining: 41.1s
697:	total: 1m 34s	remaining: 40.9s
698:	total: 1m 34s	remaining: 40.8s
699:	total: 1m 34s	remaining: 40.7s
700:	total: 1m 35s	remaining: 40.5s
701:	total: 1m 35s	remaining: 40.4s
702:	total: 1m 35s	remaining: 40.3s
703:	total: 1m 35s	remaining: 40.1s
704:	total: 1m 35s	remaining: 40s
705:	total: 1m 35s	remaining: 39.9s
706:	total: 1m 35s	remaining: 39.7s
707:	total: 1m 35s	remaining: 39.6s
708:	total: 1m 36s	remaining: 39.5s
709:	total: 1m 36s	remaining: 39.3s
710:	total: 1m 36s	remaining: 39.2s
711:	total: 1m 36s	remaining: 39s
712:	total: 1m 36s	remaining: 38.9s
713:	total: 1m 36s	remaining: 38.8s
714:	total: 1m 36s	remaining: 38.6s
715:	total: 1m 37s	remaining: 38.5s
716:	total: 1m 37s	remaining: 38.4s
717:	total: 1m 37s	remaining: 38.2s
718:	total: 1m 37s	remaining: 38.1s
719:	total: 1m 37s	remaining: 38s
720:	total: 1m 37s	remaining: 37.8s
721:	total: 1m 37s	remaining: 37.7s
722:	total: 1m 38s	remaining: 37.6s
723:	total: 1m 38s	remaining: 37.4

927:	total: 2m 5s	remaining: 9.77s
928:	total: 2m 6s	remaining: 9.63s
929:	total: 2m 6s	remaining: 9.5s
930:	total: 2m 6s	remaining: 9.36s
931:	total: 2m 6s	remaining: 9.22s
932:	total: 2m 6s	remaining: 9.09s
933:	total: 2m 6s	remaining: 8.95s
934:	total: 2m 6s	remaining: 8.81s
935:	total: 2m 6s	remaining: 8.68s
936:	total: 2m 7s	remaining: 8.55s
937:	total: 2m 7s	remaining: 8.41s
938:	total: 2m 7s	remaining: 8.27s
939:	total: 2m 7s	remaining: 8.14s
940:	total: 2m 7s	remaining: 8s
941:	total: 2m 7s	remaining: 7.87s
942:	total: 2m 7s	remaining: 7.73s
943:	total: 2m 8s	remaining: 7.59s
944:	total: 2m 8s	remaining: 7.46s
945:	total: 2m 8s	remaining: 7.32s
946:	total: 2m 8s	remaining: 7.19s
947:	total: 2m 8s	remaining: 7.05s
948:	total: 2m 8s	remaining: 6.92s
949:	total: 2m 8s	remaining: 6.78s
950:	total: 2m 9s	remaining: 6.65s
951:	total: 2m 9s	remaining: 6.51s
952:	total: 2m 9s	remaining: 6.38s
953:	total: 2m 9s	remaining: 6.24s
954:	total: 2m 9s	remaining: 6.11s
955:	total: 2m 9s	remain

In [7]:
y_pred = learner.predict(x_valid)
f1 = f1_score(y_valid, y_pred)

print(f'F1 = {f1:.2f}')

F1 = 0.95


In [8]:
print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.92      0.95       419
           1       0.93      0.98      0.95       419

    accuracy                           0.95       838
   macro avg       0.95      0.95      0.95       838
weighted avg       0.95      0.95      0.95       838



In [9]:
feat_imp_classifier = pd.DataFrame(np.c_[list(x_train), learner.estimator.get_feature_importance().astype('float64')], columns=['feature', 'coef'])

In [10]:
feat_imp_classifier['coef'] = feat_imp_classifier['coef'].astype('float64')

In [11]:
feat_imp_classifier.sort_values(by='coef', ascending=False).head(50)

,feature,coef
161,text_floor,29.156327
60,per_square_meter_price,12.192051
69,region,11.088747
0,floor,4.101688
2,lng,3.998484
13,osm_catering_points_in_0.0075,3.261329
75,address,3.197765
70,total_square,2.675301
76,log_total_square,2.597613
56,osm_transport_stop_closest_dist,2.023509


In [12]:
best_100_cols = feat_imp_classifier.sort_values(by='coef', ascending=False).head(100)['feature'].values

In [13]:
tresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
accuracies = []
f1_scores = []

for treshold in tresholds:
    y_probs = learner.predict_proba(x_valid)
    y_preds = np.where(y_probs[:, 1] > treshold, 1, 0)
    accuracies.append((treshold, accuracy_score(y_preds, y_valid)))
    f1_scores.append((treshold, f1_score(y_preds, y_valid)))

In [14]:
f1_scores

[(0.1, 0.8960342979635584),
 (0.2, 0.9225663716814159),
 (0.3, 0.9348314606741573),
 (0.4, 0.9461626575028638),
 (0.5, 0.9511627906976744),
 (0.6, 0.9482352941176471),
 (0.7, 0.9376498800959232),
 (0.8, 0.9079601990049752),
 (0.9, 0.7899860917941586)]

In [15]:
accuracies

[(0.1, 0.8842482100238663),
 (0.2, 0.9164677804295943),
 (0.3, 0.9307875894988067),
 (0.4, 0.9439140811455847),
 (0.5, 0.9498806682577565),
 (0.6, 0.9474940334128878),
 (0.7, 0.9379474940334129),
 (0.8, 0.9116945107398569),
 (0.9, 0.8198090692124105)]

In [16]:
predict_type_probs = learner.predict_proba(data_to_predict_type[cols])
predict_type_preds = np.where(predict_type_probs[:, 1] > 0.85, 1, 0)

In [17]:
ind_predicted_1_type = np.where(predict_type_preds == 1)[0]
ind_predicted_0_type = np.where(predict_type_preds == 0)[0]

In [18]:
predicted_data_1 = data_to_predict_type.iloc[ind_predicted_1_type]
predicted_data_1['price_type'] = 1
# shape_predicted_type_1 = predicted_data_1.shape[0]
# predicted_data_0 = data_to_predict_type.iloc[ind_predicted_0_type].sample(shape_predicted_type_1)

#predicted_data = pd.concat([predicted_data_0, predicted_data_1], axis=0)

C:\Users\tuman\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
def vectorize_text_features(train_data, test_data, col):
    vectorizer = TfidfVectorizer(ngram_range=(1, 2), analyzer='char', max_features=50)
    
    train_text_data = vectorizer.fit_transform(train_data[col])
    test_text_data = vectorizer.transform(test_data[col])
    
    train_text_df = pd.DataFrame(data=train_text_data.toarray(), columns=list(vectorizer.vocabulary_))
    test_text_df = pd.DataFrame(data=test_text_data.toarray(), columns=list(vectorizer.vocabulary_))
    
    train_text_df.reset_index(inplace=True, drop=True)
    test_text_df.reset_index(inplace=True, drop=True)
    
    train_data = pd.concat([train_data, train_text_df], axis=1)
    test_data = pd.concat([test_data, test_text_df], axis=1)
    
    train_data.drop([col], axis=1, inplace=True)
    test_data.drop([col], axis=1, inplace=True)
    
    return train_data, test_data

test_data['address'].fillna(method='backfill', inplace=True)
predicted_data_1.reset_index(inplace=True, drop=True)
predicted_data, test_data = vectorize_text_features(predicted_data_1, test_data, 'address')
#predicted_data, test_data = vectorize_text_features(predicted_data, test_data, 'text_floor')

In [20]:
cols_pr = [col for col in list(predicted_data) if col != 'id' and 'per_square_meter_price' not in col and 'address' not in col and 'text_floor' not in col and 'price_type' not in col]

pr_x_train, pr_x_test, pr_y_train, pr_y_test = train_test_split(predicted_data[cols_pr], 
                                                                predicted_data['per_square_meter_price'], 
                                                                test_size=0.2, 
                                                                random_state=42)


catboost_regressor = CatBoostRegressor(
    loss_function='RMSE',
    eval_metric='R2',
    task_type='CPU',
    cat_features=['osm_city_nearest_name', 
                  'day', 
                  'realty_type', 
                  'region']
)

catboost_regressor.fit(pr_x_train, pr_y_train)

pr_y_pred = catboost_regressor.predict(pr_x_test)
r2 = r2_score(pr_y_test, pr_y_pred)
mae = mean_absolute_error(pr_y_test, pr_y_pred)
mse = mean_squared_error(pr_y_test, pr_y_pred)
rmse = np.sqrt(mse)

print(f'R2 = {r2:.2f} | MAE = {mae:.0f} | RMSE = {rmse:.0f}')

Learning rate set to 0.054231
0:	learn: 0.0294780	total: 102ms	remaining: 1m 41s
1:	learn: 0.0581260	total: 210ms	remaining: 1m 44s
2:	learn: 0.0815446	total: 305ms	remaining: 1m 41s
3:	learn: 0.1043978	total: 395ms	remaining: 1m 38s
4:	learn: 0.1254217	total: 507ms	remaining: 1m 40s
5:	learn: 0.1430209	total: 592ms	remaining: 1m 38s
6:	learn: 0.1660897	total: 687ms	remaining: 1m 37s
7:	learn: 0.1873027	total: 773ms	remaining: 1m 35s
8:	learn: 0.2018020	total: 856ms	remaining: 1m 34s
9:	learn: 0.2216101	total: 941ms	remaining: 1m 33s
10:	learn: 0.2384592	total: 1.03s	remaining: 1m 33s
11:	learn: 0.2522925	total: 1.13s	remaining: 1m 32s
12:	learn: 0.2685791	total: 1.23s	remaining: 1m 33s
13:	learn: 0.2842686	total: 1.31s	remaining: 1m 32s
14:	learn: 0.2960817	total: 1.42s	remaining: 1m 33s
15:	learn: 0.3101321	total: 1.52s	remaining: 1m 33s
16:	learn: 0.3169299	total: 1.61s	remaining: 1m 33s
17:	learn: 0.3247810	total: 1.71s	remaining: 1m 33s
18:	learn: 0.3331367	total: 1.82s	remaining:

157:	learn: 0.6410091	total: 15.5s	remaining: 1m 22s
158:	learn: 0.6417816	total: 15.6s	remaining: 1m 22s
159:	learn: 0.6423155	total: 15.7s	remaining: 1m 22s
160:	learn: 0.6434282	total: 15.8s	remaining: 1m 22s
161:	learn: 0.6459168	total: 15.9s	remaining: 1m 22s
162:	learn: 0.6479609	total: 16s	remaining: 1m 22s
163:	learn: 0.6490554	total: 16.1s	remaining: 1m 22s
164:	learn: 0.6514151	total: 16.2s	remaining: 1m 21s
165:	learn: 0.6541745	total: 16.3s	remaining: 1m 21s
166:	learn: 0.6544006	total: 16.4s	remaining: 1m 21s
167:	learn: 0.6550673	total: 16.5s	remaining: 1m 21s
168:	learn: 0.6554581	total: 16.5s	remaining: 1m 21s
169:	learn: 0.6556200	total: 16.6s	remaining: 1m 21s
170:	learn: 0.6578550	total: 16.7s	remaining: 1m 21s
171:	learn: 0.6588316	total: 16.8s	remaining: 1m 20s
172:	learn: 0.6592273	total: 16.9s	remaining: 1m 20s
173:	learn: 0.6611180	total: 17s	remaining: 1m 20s
174:	learn: 0.6632629	total: 17.1s	remaining: 1m 20s
175:	learn: 0.6638891	total: 17.2s	remaining: 1m 2

313:	learn: 0.7631046	total: 30.7s	remaining: 1m 6s
314:	learn: 0.7635405	total: 30.8s	remaining: 1m 6s
315:	learn: 0.7638952	total: 30.9s	remaining: 1m 6s
316:	learn: 0.7640675	total: 31s	remaining: 1m 6s
317:	learn: 0.7644299	total: 31.1s	remaining: 1m 6s
318:	learn: 0.7655550	total: 31.2s	remaining: 1m 6s
319:	learn: 0.7658806	total: 31.3s	remaining: 1m 6s
320:	learn: 0.7662081	total: 31.4s	remaining: 1m 6s
321:	learn: 0.7666936	total: 31.4s	remaining: 1m 6s
322:	learn: 0.7675603	total: 31.5s	remaining: 1m 6s
323:	learn: 0.7680333	total: 31.6s	remaining: 1m 5s
324:	learn: 0.7680967	total: 31.7s	remaining: 1m 5s
325:	learn: 0.7684875	total: 31.9s	remaining: 1m 5s
326:	learn: 0.7688791	total: 32s	remaining: 1m 5s
327:	learn: 0.7689040	total: 32.1s	remaining: 1m 5s
328:	learn: 0.7690560	total: 32.2s	remaining: 1m 5s
329:	learn: 0.7692035	total: 32.3s	remaining: 1m 5s
330:	learn: 0.7694980	total: 32.4s	remaining: 1m 5s
331:	learn: 0.7703905	total: 32.5s	remaining: 1m 5s
332:	learn: 0.77

473:	learn: 0.8242442	total: 46.2s	remaining: 51.3s
474:	learn: 0.8244925	total: 46.3s	remaining: 51.2s
475:	learn: 0.8248672	total: 46.4s	remaining: 51.1s
476:	learn: 0.8248947	total: 46.6s	remaining: 51s
477:	learn: 0.8251388	total: 46.7s	remaining: 51s
478:	learn: 0.8251595	total: 46.8s	remaining: 50.9s
479:	learn: 0.8255202	total: 46.9s	remaining: 50.8s
480:	learn: 0.8255610	total: 47s	remaining: 50.7s
481:	learn: 0.8257323	total: 47.1s	remaining: 50.6s
482:	learn: 0.8257469	total: 47.2s	remaining: 50.5s
483:	learn: 0.8260582	total: 47.3s	remaining: 50.4s
484:	learn: 0.8262840	total: 47.4s	remaining: 50.3s
485:	learn: 0.8263776	total: 47.5s	remaining: 50.2s
486:	learn: 0.8266197	total: 47.6s	remaining: 50.1s
487:	learn: 0.8269380	total: 47.7s	remaining: 50s
488:	learn: 0.8274122	total: 47.8s	remaining: 49.9s
489:	learn: 0.8275392	total: 47.9s	remaining: 49.8s
490:	learn: 0.8277361	total: 48s	remaining: 49.7s
491:	learn: 0.8279016	total: 48.1s	remaining: 49.6s
492:	learn: 0.8284996	

634:	learn: 0.8610925	total: 1m 2s	remaining: 35.7s
635:	learn: 0.8613177	total: 1m 2s	remaining: 35.6s
636:	learn: 0.8613752	total: 1m 2s	remaining: 35.5s
637:	learn: 0.8615833	total: 1m 2s	remaining: 35.4s
638:	learn: 0.8615928	total: 1m 2s	remaining: 35.3s
639:	learn: 0.8617348	total: 1m 2s	remaining: 35.2s
640:	learn: 0.8618822	total: 1m 2s	remaining: 35.1s
641:	learn: 0.8621621	total: 1m 2s	remaining: 35s
642:	learn: 0.8621691	total: 1m 2s	remaining: 34.9s
643:	learn: 0.8623095	total: 1m 3s	remaining: 34.8s
644:	learn: 0.8624487	total: 1m 3s	remaining: 34.7s
645:	learn: 0.8625125	total: 1m 3s	remaining: 34.6s
646:	learn: 0.8627676	total: 1m 3s	remaining: 34.5s
647:	learn: 0.8629467	total: 1m 3s	remaining: 34.4s
648:	learn: 0.8631006	total: 1m 3s	remaining: 34.4s
649:	learn: 0.8632918	total: 1m 3s	remaining: 34.3s
650:	learn: 0.8634634	total: 1m 3s	remaining: 34.2s
651:	learn: 0.8636353	total: 1m 3s	remaining: 34.1s
652:	learn: 0.8637478	total: 1m 3s	remaining: 34s
653:	learn: 0.86

791:	learn: 0.8836142	total: 1m 17s	remaining: 20.3s
792:	learn: 0.8837643	total: 1m 17s	remaining: 20.2s
793:	learn: 0.8838304	total: 1m 17s	remaining: 20.1s
794:	learn: 0.8839532	total: 1m 17s	remaining: 20s
795:	learn: 0.8841588	total: 1m 17s	remaining: 19.9s
796:	learn: 0.8842914	total: 1m 17s	remaining: 19.9s
797:	learn: 0.8843805	total: 1m 18s	remaining: 19.7s
798:	learn: 0.8845496	total: 1m 18s	remaining: 19.7s
799:	learn: 0.8847909	total: 1m 18s	remaining: 19.6s
800:	learn: 0.8848306	total: 1m 18s	remaining: 19.5s
801:	learn: 0.8849811	total: 1m 18s	remaining: 19.4s
802:	learn: 0.8849848	total: 1m 18s	remaining: 19.3s
803:	learn: 0.8851495	total: 1m 18s	remaining: 19.2s
804:	learn: 0.8853501	total: 1m 18s	remaining: 19.1s
805:	learn: 0.8855357	total: 1m 18s	remaining: 19s
806:	learn: 0.8855399	total: 1m 18s	remaining: 18.9s
807:	learn: 0.8856108	total: 1m 19s	remaining: 18.8s
808:	learn: 0.8857352	total: 1m 19s	remaining: 18.7s
809:	learn: 0.8857886	total: 1m 19s	remaining: 18.

946:	learn: 0.9018937	total: 1m 32s	remaining: 5.17s
947:	learn: 0.9019692	total: 1m 32s	remaining: 5.08s
948:	learn: 0.9020409	total: 1m 32s	remaining: 4.98s
949:	learn: 0.9022268	total: 1m 32s	remaining: 4.88s
950:	learn: 0.9022847	total: 1m 32s	remaining: 4.78s
951:	learn: 0.9023569	total: 1m 32s	remaining: 4.68s
952:	learn: 0.9024915	total: 1m 32s	remaining: 4.59s
953:	learn: 0.9025185	total: 1m 33s	remaining: 4.49s
954:	learn: 0.9026107	total: 1m 33s	remaining: 4.39s
955:	learn: 0.9027556	total: 1m 33s	remaining: 4.29s
956:	learn: 0.9028611	total: 1m 33s	remaining: 4.2s
957:	learn: 0.9029961	total: 1m 33s	remaining: 4.1s
958:	learn: 0.9031257	total: 1m 33s	remaining: 4s
959:	learn: 0.9033033	total: 1m 33s	remaining: 3.9s
960:	learn: 0.9034444	total: 1m 33s	remaining: 3.81s
961:	learn: 0.9035090	total: 1m 33s	remaining: 3.71s
962:	learn: 0.9035475	total: 1m 34s	remaining: 3.61s
963:	learn: 0.9036896	total: 1m 34s	remaining: 3.52s
964:	learn: 0.9037088	total: 1m 34s	remaining: 3.42s

In [21]:
import sys
sys.path.append('./raif/notebooks/baseline/raifhack_ds/')

from metrics import metrics_stat, deviation_metric


raif_metric = metrics_stat(np.array(pr_y_test), np.array(pr_y_pred))['raif_metric']
print(f'Raif = {raif_metric}')

Raif = 1.2875301742238785


In [ ]:
test_pred = model.predict(test_data[cols_pr])
pd.DataFrame(np.c_[test_data['id'], test_pred], columns=['id', 'per_square_meter_price']).set_index('id').to_csv('submission.csv')

In [47]:
train_data_1 = train_data[train_data['price_type'] == 1]
test_data = pd.read_csv('./preprocessed_data/test_data.csv')

test_data['address'].fillna(method='backfill', inplace=True)

train_data_1.reset_index(inplace=True, drop=True)
#train_data_1, test_data = vectorize_text_features(train_data_1, test_data, 'address')
train_data_1['per_square_meter_price'] = np.log(train_data_1['per_square_meter_price'])

cols_pr = [col for col in list(train_data_1) if col != 'id' and 'price_eur_rub' not in col and 'price_usd_rub' not in col and 'per_square_meter_price' not in col and 'price_type' not in col]

pr_x_train, pr_x_test, pr_y_train, pr_y_test = train_test_split(train_data_1[cols_pr], 
                                                                train_data_1['per_square_meter_price'], 
                                                                test_size=0.2, 
                                                                random_state=42)

model = CatBoostRegressor(
    loss_function='RMSE',
    eval_metric='R2',
    task_type='CPU',
    cat_features=['osm_city_nearest_name', 
                  'day', 
                  'realty_type', 
                  'region',
                  'address',
                  'text_floor']
)

model.fit(pr_x_train, pr_y_train)

pr_y_pred = model.predict(pr_x_test)
r2 = r2_score(np.e**pr_y_test, np.e**pr_y_pred)
mae = mean_absolute_error(np.e**pr_y_test, np.e**pr_y_pred)
mse = mean_squared_error(np.e**pr_y_test, np.e**pr_y_pred)
rmse = np.sqrt(mse)

print(f'R2 = {r2:.2f} | MAE = {mae:.0f} | RMSE = {rmse:.0f}')

C:\Users\tuman\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Learning rate set to 0.048396
0:	learn: 0.0370174	total: 72.1ms	remaining: 1m 12s
1:	learn: 0.0714846	total: 140ms	remaining: 1m 10s
2:	learn: 0.1015439	total: 218ms	remaining: 1m 12s
3:	learn: 0.1310252	total: 294ms	remaining: 1m 13s
4:	learn: 0.1587377	total: 388ms	remaining: 1m 17s
5:	learn: 0.1840231	total: 473ms	remaining: 1m 18s
6:	learn: 0.2088486	total: 561ms	remaining: 1m 19s
7:	learn: 0.2308686	total: 665ms	remaining: 1m 22s
8:	learn: 0.2506177	total: 774ms	remaining: 1m 25s
9:	learn: 0.2685417	total: 859ms	remaining: 1m 25s
10:	learn: 0.2873353	total: 959ms	remaining: 1m 26s
11:	learn: 0.3020414	total: 1.06s	remaining: 1m 27s
12:	learn: 0.3161466	total: 1.16s	remaining: 1m 27s
13:	learn: 0.3291405	total: 1.25s	remaining: 1m 28s
14:	learn: 0.3425823	total: 1.33s	remaining: 1m 27s
15:	learn: 0.3544285	total: 1.41s	remaining: 1m 26s
16:	learn: 0.3672132	total: 1.5s	remaining: 1m 26s
17:	learn: 0.3776360	total: 1.58s	remaining: 1m 26s
18:	learn: 0.3873481	total: 1.66s	remaining:

159:	learn: 0.6577122	total: 14.5s	remaining: 1m 16s
160:	learn: 0.6586620	total: 14.6s	remaining: 1m 16s
161:	learn: 0.6592239	total: 14.7s	remaining: 1m 16s
162:	learn: 0.6595369	total: 14.8s	remaining: 1m 16s
163:	learn: 0.6599991	total: 14.9s	remaining: 1m 15s
164:	learn: 0.6603588	total: 15s	remaining: 1m 15s
165:	learn: 0.6614378	total: 15.1s	remaining: 1m 15s
166:	learn: 0.6619867	total: 15.1s	remaining: 1m 15s
167:	learn: 0.6622492	total: 15.2s	remaining: 1m 15s
168:	learn: 0.6634922	total: 15.3s	remaining: 1m 15s
169:	learn: 0.6642590	total: 15.4s	remaining: 1m 15s
170:	learn: 0.6647695	total: 15.5s	remaining: 1m 15s
171:	learn: 0.6652691	total: 15.6s	remaining: 1m 15s
172:	learn: 0.6659934	total: 15.7s	remaining: 1m 14s
173:	learn: 0.6662753	total: 15.8s	remaining: 1m 14s
174:	learn: 0.6667990	total: 15.8s	remaining: 1m 14s
175:	learn: 0.6672185	total: 15.9s	remaining: 1m 14s
176:	learn: 0.6675247	total: 16s	remaining: 1m 14s
177:	learn: 0.6680451	total: 16.1s	remaining: 1m 1

318:	learn: 0.7352414	total: 28.5s	remaining: 1m
319:	learn: 0.7355972	total: 28.6s	remaining: 1m
320:	learn: 0.7361816	total: 28.6s	remaining: 1m
321:	learn: 0.7362494	total: 28.7s	remaining: 1m
322:	learn: 0.7365628	total: 28.8s	remaining: 1m
323:	learn: 0.7366161	total: 28.9s	remaining: 1m
324:	learn: 0.7368242	total: 29s	remaining: 1m
325:	learn: 0.7369156	total: 29.1s	remaining: 1m
326:	learn: 0.7375797	total: 29.1s	remaining: 60s
327:	learn: 0.7376328	total: 29.2s	remaining: 59.9s
328:	learn: 0.7381615	total: 29.3s	remaining: 59.8s
329:	learn: 0.7384677	total: 29.4s	remaining: 59.7s
330:	learn: 0.7390667	total: 29.5s	remaining: 59.6s
331:	learn: 0.7394319	total: 29.6s	remaining: 59.5s
332:	learn: 0.7402022	total: 29.7s	remaining: 59.5s
333:	learn: 0.7405187	total: 29.8s	remaining: 59.4s
334:	learn: 0.7407682	total: 29.9s	remaining: 59.3s
335:	learn: 0.7410499	total: 30s	remaining: 59.2s
336:	learn: 0.7412173	total: 30.1s	remaining: 59.2s
337:	learn: 0.7416571	total: 30.2s	remaini

480:	learn: 0.7857951	total: 43.4s	remaining: 46.8s
481:	learn: 0.7858670	total: 43.4s	remaining: 46.7s
482:	learn: 0.7862116	total: 43.5s	remaining: 46.6s
483:	learn: 0.7865725	total: 43.6s	remaining: 46.5s
484:	learn: 0.7868220	total: 43.7s	remaining: 46.4s
485:	learn: 0.7870091	total: 43.8s	remaining: 46.3s
486:	learn: 0.7871289	total: 43.9s	remaining: 46.2s
487:	learn: 0.7871726	total: 43.9s	remaining: 46.1s
488:	learn: 0.7875823	total: 44s	remaining: 46s
489:	learn: 0.7876169	total: 44.1s	remaining: 45.9s
490:	learn: 0.7880680	total: 44.2s	remaining: 45.8s
491:	learn: 0.7884709	total: 44.3s	remaining: 45.7s
492:	learn: 0.7886314	total: 44.4s	remaining: 45.6s
493:	learn: 0.7887158	total: 44.5s	remaining: 45.5s
494:	learn: 0.7888133	total: 44.5s	remaining: 45.4s
495:	learn: 0.7888483	total: 44.6s	remaining: 45.3s
496:	learn: 0.7892872	total: 44.7s	remaining: 45.2s
497:	learn: 0.7893679	total: 44.8s	remaining: 45.1s
498:	learn: 0.7896164	total: 44.9s	remaining: 45s
499:	learn: 0.7897

640:	learn: 0.8192429	total: 57.6s	remaining: 32.3s
641:	learn: 0.8192606	total: 57.7s	remaining: 32.2s
642:	learn: 0.8195715	total: 57.8s	remaining: 32.1s
643:	learn: 0.8196681	total: 57.9s	remaining: 32s
644:	learn: 0.8197349	total: 58s	remaining: 31.9s
645:	learn: 0.8201001	total: 58.1s	remaining: 31.8s
646:	learn: 0.8202430	total: 58.2s	remaining: 31.8s
647:	learn: 0.8203000	total: 58.3s	remaining: 31.7s
648:	learn: 0.8204447	total: 58.4s	remaining: 31.6s
649:	learn: 0.8205744	total: 58.5s	remaining: 31.5s
650:	learn: 0.8206722	total: 58.6s	remaining: 31.4s
651:	learn: 0.8210245	total: 58.7s	remaining: 31.3s
652:	learn: 0.8210769	total: 58.8s	remaining: 31.2s
653:	learn: 0.8211891	total: 58.9s	remaining: 31.1s
654:	learn: 0.8216564	total: 58.9s	remaining: 31s
655:	learn: 0.8218712	total: 59s	remaining: 31s
656:	learn: 0.8220737	total: 59.1s	remaining: 30.9s
657:	learn: 0.8222402	total: 59.2s	remaining: 30.8s
658:	learn: 0.8224690	total: 59.3s	remaining: 30.7s
659:	learn: 0.8226358	

799:	learn: 0.8479627	total: 1m 12s	remaining: 18s
800:	learn: 0.8480420	total: 1m 12s	remaining: 17.9s
801:	learn: 0.8483414	total: 1m 12s	remaining: 17.9s
802:	learn: 0.8483457	total: 1m 12s	remaining: 17.8s
803:	learn: 0.8485705	total: 1m 12s	remaining: 17.7s
804:	learn: 0.8486205	total: 1m 12s	remaining: 17.6s
805:	learn: 0.8487459	total: 1m 12s	remaining: 17.5s
806:	learn: 0.8488640	total: 1m 12s	remaining: 17.4s
807:	learn: 0.8489747	total: 1m 12s	remaining: 17.3s
808:	learn: 0.8490239	total: 1m 12s	remaining: 17.2s
809:	learn: 0.8491803	total: 1m 13s	remaining: 17.1s
810:	learn: 0.8491883	total: 1m 13s	remaining: 17s
811:	learn: 0.8493331	total: 1m 13s	remaining: 17s
812:	learn: 0.8495568	total: 1m 13s	remaining: 16.9s
813:	learn: 0.8496773	total: 1m 13s	remaining: 16.8s
814:	learn: 0.8500114	total: 1m 13s	remaining: 16.7s
815:	learn: 0.8501544	total: 1m 13s	remaining: 16.6s
816:	learn: 0.8503177	total: 1m 13s	remaining: 16.5s
817:	learn: 0.8504990	total: 1m 13s	remaining: 16.4s

956:	learn: 0.8695992	total: 1m 26s	remaining: 3.91s
957:	learn: 0.8697491	total: 1m 27s	remaining: 3.81s
958:	learn: 0.8699444	total: 1m 27s	remaining: 3.72s
959:	learn: 0.8701990	total: 1m 27s	remaining: 3.63s
960:	learn: 0.8703745	total: 1m 27s	remaining: 3.54s
961:	learn: 0.8705718	total: 1m 27s	remaining: 3.45s
962:	learn: 0.8707152	total: 1m 27s	remaining: 3.36s
963:	learn: 0.8707843	total: 1m 27s	remaining: 3.27s
964:	learn: 0.8708868	total: 1m 27s	remaining: 3.18s
965:	learn: 0.8709779	total: 1m 27s	remaining: 3.09s
966:	learn: 0.8710146	total: 1m 27s	remaining: 3s
967:	learn: 0.8711690	total: 1m 28s	remaining: 2.91s
968:	learn: 0.8712150	total: 1m 28s	remaining: 2.82s
969:	learn: 0.8716052	total: 1m 28s	remaining: 2.73s
970:	learn: 0.8716876	total: 1m 28s	remaining: 2.64s
971:	learn: 0.8718784	total: 1m 28s	remaining: 2.55s
972:	learn: 0.8719533	total: 1m 28s	remaining: 2.46s
973:	learn: 0.8720108	total: 1m 28s	remaining: 2.37s
974:	learn: 0.8722480	total: 1m 28s	remaining: 2.

In [48]:
from metrics import metrics_stat, deviation_metric

raif_metric = metrics_stat(np.e**np.array(pr_y_test), np.e**np.array(pr_y_pred))['raif_metric']
print(f'Raif = {raif_metric}')

Raif = 1.338337684003492


In [497]:
test_pred = model.predict(test_data[cols_pr])
pd.DataFrame(np.c_[test_data['id'], test_pred], columns=['id', 'per_square_meter_price']).set_index('id').to_csv('test_submission_3.csv')

In [7]:
kf = KFold(3)

cols_pr = [col for col in list(train_data_1) if col != 'id' not in col and 'per_square_meter_price' not in col and 'price_type' not in col]

catboost_params = {
                    'iterations': np.arange(100, 1100, 100),
                    'depth': np.arange(6, 16, 2),
                    'learning_rate': np.arange(0.01, 1, 0.02),
                    'random_strength': np.logspace(-9, 0, 10),
                    'bagging_temperature': np.arange(0.1, 2.1, 0.1),
                    'border_count': np.arange(1, 255, 1),
                    'l2_leaf_reg': np.arange(1, 19, 1),
                    'min_data_in_leaf': np.arange(1, 7, 1)
}

catboost = CatBoostRegressor(
    loss_function='RMSE',
    eval_metric='R2',
    task_type='CPU',
    cat_features=['osm_city_nearest_name', 
                  'day', 
                  'realty_type', 
                  'region',
                  'address',
                  'text_floor']
)

opt_catboost = BayesSearchCV(catboost, catboost_params, cv=kf, n_iter=100, scoring='r2', random_state=42)
opt_catboost.fit(train_data_1[cols_pr], train_data_1['per_square_meter_price'], callback=[VerboseCallback(100), DeadlineStopper(60*300), DeltaYStopper(0.001, 5)])

Iteration No: 1 started. Searching for the next optimal point.
0:	learn: 0.0882786	total: 2.19s	remaining: 14m 35s
1:	learn: 0.1978825	total: 4.43s	remaining: 14m 41s
2:	learn: 0.2843990	total: 6.63s	remaining: 14m 37s
3:	learn: 0.3690632	total: 8.81s	remaining: 14m 32s
4:	learn: 0.4363834	total: 11s	remaining: 14m 32s
5:	learn: 0.4815992	total: 13.5s	remaining: 14m 43s
6:	learn: 0.5156905	total: 15.8s	remaining: 14m 49s
7:	learn: 0.5506609	total: 18.2s	remaining: 14m 51s
8:	learn: 0.6023336	total: 20.6s	remaining: 14m 54s
9:	learn: 0.6369070	total: 23s	remaining: 14m 55s
10:	learn: 0.6647660	total: 25.3s	remaining: 14m 54s
11:	learn: 0.6872952	total: 27.5s	remaining: 14m 49s
12:	learn: 0.7106009	total: 29.7s	remaining: 14m 45s
13:	learn: 0.7345996	total: 32.1s	remaining: 14m 45s
14:	learn: 0.7554984	total: 34.5s	remaining: 14m 45s
15:	learn: 0.7726608	total: 37s	remaining: 14m 48s
16:	learn: 0.7872486	total: 39.5s	remaining: 14m 50s
17:	learn: 0.7984848	total: 41.9s	remaining: 14m 48s

152:	learn: 0.9981420	total: 6m 3s	remaining: 9m 47s
153:	learn: 0.9981900	total: 6m 6s	remaining: 9m 44s
154:	learn: 0.9982329	total: 6m 8s	remaining: 9m 42s
155:	learn: 0.9983102	total: 6m 10s	remaining: 9m 39s
156:	learn: 0.9983658	total: 6m 12s	remaining: 9m 37s
157:	learn: 0.9983865	total: 6m 15s	remaining: 9m 34s
158:	learn: 0.9984269	total: 6m 17s	remaining: 9m 32s
159:	learn: 0.9984914	total: 6m 19s	remaining: 9m 29s
160:	learn: 0.9984978	total: 6m 21s	remaining: 9m 27s
161:	learn: 0.9985599	total: 6m 24s	remaining: 9m 24s
162:	learn: 0.9985726	total: 6m 26s	remaining: 9m 22s
163:	learn: 0.9985754	total: 6m 29s	remaining: 9m 19s
164:	learn: 0.9986389	total: 6m 31s	remaining: 9m 17s
165:	learn: 0.9986807	total: 6m 33s	remaining: 9m 15s
166:	learn: 0.9986918	total: 6m 36s	remaining: 9m 12s
167:	learn: 0.9987461	total: 6m 38s	remaining: 9m 10s
168:	learn: 0.9987968	total: 6m 40s	remaining: 9m 7s
169:	learn: 0.9988017	total: 6m 41s	remaining: 9m 3s
170:	learn: 0.9988571	total: 6m 4

305:	learn: 0.9999423	total: 11m 54s	remaining: 3m 39s
306:	learn: 0.9999425	total: 11m 57s	remaining: 3m 37s
307:	learn: 0.9999434	total: 11m 59s	remaining: 3m 34s
308:	learn: 0.9999451	total: 12m 1s	remaining: 3m 32s
309:	learn: 0.9999458	total: 12m 4s	remaining: 3m 30s
310:	learn: 0.9999458	total: 12m 6s	remaining: 3m 27s
311:	learn: 0.9999479	total: 12m 8s	remaining: 3m 25s
312:	learn: 0.9999495	total: 12m 11s	remaining: 3m 23s
313:	learn: 0.9999516	total: 12m 13s	remaining: 3m 20s
314:	learn: 0.9999517	total: 12m 15s	remaining: 3m 18s
315:	learn: 0.9999528	total: 12m 18s	remaining: 3m 16s
316:	learn: 0.9999529	total: 12m 20s	remaining: 3m 13s
317:	learn: 0.9999529	total: 12m 22s	remaining: 3m 11s
318:	learn: 0.9999538	total: 12m 25s	remaining: 3m 9s
319:	learn: 0.9999560	total: 12m 27s	remaining: 3m 6s
320:	learn: 0.9999560	total: 12m 29s	remaining: 3m 4s
321:	learn: 0.9999560	total: 12m 32s	remaining: 3m 2s
322:	learn: 0.9999560	total: 12m 34s	remaining: 2m 59s
323:	learn: 0.9999

58:	learn: 0.9227720	total: 2m 15s	remaining: 13m
59:	learn: 0.9252660	total: 2m 17s	remaining: 12m 59s
60:	learn: 0.9272001	total: 2m 19s	remaining: 12m 57s
61:	learn: 0.9275232	total: 2m 22s	remaining: 12m 54s
62:	learn: 0.9289775	total: 2m 24s	remaining: 12m 52s
63:	learn: 0.9328926	total: 2m 26s	remaining: 12m 50s
64:	learn: 0.9332546	total: 2m 28s	remaining: 12m 47s
65:	learn: 0.9369643	total: 2m 31s	remaining: 12m 45s
66:	learn: 0.9391990	total: 2m 33s	remaining: 12m 42s
67:	learn: 0.9393214	total: 2m 35s	remaining: 12m 40s
68:	learn: 0.9401085	total: 2m 38s	remaining: 12m 38s
69:	learn: 0.9419754	total: 2m 40s	remaining: 12m 35s
70:	learn: 0.9428803	total: 2m 42s	remaining: 12m 33s
71:	learn: 0.9429375	total: 2m 44s	remaining: 12m 31s
72:	learn: 0.9430432	total: 2m 47s	remaining: 12m 29s
73:	learn: 0.9453239	total: 2m 49s	remaining: 12m 26s
74:	learn: 0.9471025	total: 2m 51s	remaining: 12m 24s
75:	learn: 0.9484305	total: 2m 54s	remaining: 12m 21s
76:	learn: 0.9500036	total: 2m 5

211:	learn: 0.9969577	total: 8m 19s	remaining: 7m 23s
212:	learn: 0.9970193	total: 8m 22s	remaining: 7m 20s
213:	learn: 0.9970799	total: 8m 24s	remaining: 7m 18s
214:	learn: 0.9972255	total: 8m 27s	remaining: 7m 16s
215:	learn: 0.9973703	total: 8m 30s	remaining: 7m 14s
216:	learn: 0.9974431	total: 8m 32s	remaining: 7m 12s
217:	learn: 0.9974899	total: 8m 35s	remaining: 7m 10s
218:	learn: 0.9975999	total: 8m 37s	remaining: 7m 7s
219:	learn: 0.9976011	total: 8m 40s	remaining: 7m 5s
220:	learn: 0.9977289	total: 8m 42s	remaining: 7m 3s
221:	learn: 0.9977603	total: 8m 44s	remaining: 7m
222:	learn: 0.9977615	total: 8m 46s	remaining: 6m 57s
223:	learn: 0.9978595	total: 8m 48s	remaining: 6m 55s
224:	learn: 0.9979166	total: 8m 51s	remaining: 6m 53s
225:	learn: 0.9979648	total: 8m 53s	remaining: 6m 51s
226:	learn: 0.9980210	total: 8m 56s	remaining: 6m 48s
227:	learn: 0.9980224	total: 8m 58s	remaining: 6m 46s
228:	learn: 0.9980261	total: 9m 1s	remaining: 6m 44s
229:	learn: 0.9980490	total: 9m 3s	r

...

441:	learn: 0.8005297	total: 26.1s	remaining: 33s
442:	learn: 0.8008026	total: 26.2s	remaining: 32.9s
443:	learn: 0.8010647	total: 26.2s	remaining: 32.8s
444:	learn: 0.8013680	total: 26.3s	remaining: 32.8s
445:	learn: 0.8018781	total: 26.3s	remaining: 32.7s
446:	learn: 0.8021300	total: 26.4s	remaining: 32.6s
447:	learn: 0.8023098	total: 26.4s	remaining: 32.6s
448:	learn: 0.8024064	total: 26.5s	remaining: 32.5s
449:	learn: 0.8027261	total: 26.6s	remaining: 32.5s
450:	learn: 0.8032284	total: 26.6s	remaining: 32.4s
451:	learn: 0.8033579	total: 26.7s	remaining: 32.3s
452:	learn: 0.8035493	total: 26.7s	remaining: 32.3s
453:	learn: 0.8038329	total: 26.8s	remaining: 32.2s
454:	learn: 0.8043082	total: 26.9s	remaining: 32.2s
455:	learn: 0.8045438	total: 26.9s	remaining: 32.1s
456:	learn: 0.8048064	total: 27s	remaining: 32.1s
457:	learn: 0.8052741	total: 27s	remaining: 32s
458:	learn: 0.8056086	total: 27.1s	remaining: 31.9s
459:	learn: 0.8058347	total: 27.2s	remaining: 31.9s
460:	learn: 0.806038

603:	learn: 0.8413620	total: 35.7s	remaining: 23.4s
604:	learn: 0.8415855	total: 35.7s	remaining: 23.3s
605:	learn: 0.8417780	total: 35.8s	remaining: 23.3s
606:	learn: 0.8418802	total: 35.9s	remaining: 23.2s
607:	learn: 0.8421621	total: 35.9s	remaining: 23.2s
608:	learn: 0.8423735	total: 36s	remaining: 23.1s
609:	learn: 0.8427681	total: 36s	remaining: 23s
610:	learn: 0.8429863	total: 36.1s	remaining: 23s
611:	learn: 0.8432266	total: 36.1s	remaining: 22.9s
612:	learn: 0.8434063	total: 36.2s	remaining: 22.9s
613:	learn: 0.8435796	total: 36.3s	remaining: 22.8s
614:	learn: 0.8438207	total: 36.3s	remaining: 22.7s
615:	learn: 0.8440575	total: 36.4s	remaining: 22.7s
616:	learn: 0.8443239	total: 36.5s	remaining: 22.6s
617:	learn: 0.8444902	total: 36.5s	remaining: 22.6s
618:	learn: 0.8447231	total: 36.6s	remaining: 22.5s
619:	learn: 0.8449634	total: 36.6s	remaining: 22.5s
620:	learn: 0.8451927	total: 36.7s	remaining: 22.4s
621:	learn: 0.8453476	total: 36.8s	remaining: 22.3s
622:	learn: 0.845564

764:	learn: 0.8671827	total: 45.3s	remaining: 13.9s
765:	learn: 0.8671894	total: 45.3s	remaining: 13.8s
766:	learn: 0.8675528	total: 45.4s	remaining: 13.8s
767:	learn: 0.8676476	total: 45.4s	remaining: 13.7s
768:	learn: 0.8678434	total: 45.5s	remaining: 13.7s
769:	learn: 0.8679355	total: 45.5s	remaining: 13.6s
770:	learn: 0.8680286	total: 45.6s	remaining: 13.5s
771:	learn: 0.8682517	total: 45.7s	remaining: 13.5s
772:	learn: 0.8682676	total: 45.7s	remaining: 13.4s
773:	learn: 0.8683818	total: 45.8s	remaining: 13.4s
774:	learn: 0.8685678	total: 45.8s	remaining: 13.3s
775:	learn: 0.8685851	total: 45.9s	remaining: 13.2s
776:	learn: 0.8686746	total: 46s	remaining: 13.2s
777:	learn: 0.8687386	total: 46s	remaining: 13.1s
778:	learn: 0.8688856	total: 46.1s	remaining: 13.1s
779:	learn: 0.8690003	total: 46.2s	remaining: 13s
780:	learn: 0.8692652	total: 46.2s	remaining: 13s
781:	learn: 0.8693797	total: 46.3s	remaining: 12.9s
782:	learn: 0.8694944	total: 46.3s	remaining: 12.8s
783:	learn: 0.869574

924:	learn: 0.8791973	total: 54.5s	remaining: 4.42s
925:	learn: 0.8792100	total: 54.6s	remaining: 4.36s
926:	learn: 0.8792321	total: 54.6s	remaining: 4.3s
927:	learn: 0.8793150	total: 54.7s	remaining: 4.24s
928:	learn: 0.8795059	total: 54.8s	remaining: 4.18s
929:	learn: 0.8796676	total: 54.8s	remaining: 4.13s
930:	learn: 0.8798698	total: 54.9s	remaining: 4.07s
931:	learn: 0.8799021	total: 54.9s	remaining: 4.01s
932:	learn: 0.8800198	total: 55s	remaining: 3.95s
933:	learn: 0.8802063	total: 55s	remaining: 3.89s
934:	learn: 0.8806875	total: 55.1s	remaining: 3.83s
935:	learn: 0.8807185	total: 55.2s	remaining: 3.77s
936:	learn: 0.8808893	total: 55.3s	remaining: 3.71s
937:	learn: 0.8810502	total: 55.3s	remaining: 3.66s
938:	learn: 0.8812479	total: 55.4s	remaining: 3.6s
939:	learn: 0.8814345	total: 55.4s	remaining: 3.54s
940:	learn: 0.8816180	total: 55.5s	remaining: 3.48s
941:	learn: 0.8816829	total: 55.5s	remaining: 3.42s
942:	learn: 0.8817785	total: 55.6s	remaining: 3.36s
943:	learn: 0.8818

BayesSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
              estimator=<catboost.core.CatBoostRegressor object at 0x000001769706A7C8>,
              n_iter=100, random_state=42, scoring='r2',
              search_spaces={'bagging_temperature': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. ]),
                             'border_count': array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,...
                             'learning_rate': array([0.01, 0.03, 0.05, 0.07, 0.09, 0.11, 0.13, 0.15, 0.17, 0.19, 0.21,
       0.23, 0.25, 0.27, 0.29, 0.31, 0.33, 0.35, 0.37, 0.39, 0.41, 0.43,
       0.45, 0.47, 0.49, 0.51, 0.53, 0.55, 0.57, 0.59, 0.61, 0.63, 0.65,
       0.67, 0.69, 0.71, 0.73, 0.75, 0.77, 0.79, 0.81, 0.83, 0.85, 0.87,
       0.89, 0.91, 0.93, 0.95, 0.97, 0.99]),
                             'min_data_in_leaf': array([1, 2, 3, 4, 5, 6]),
                             'random_streng